In [122]:
import os
from os import environ
from selenium import webdriver
from selenium.webdriver.support.select import Select
import dateparser
import datetime

driver = webdriver.Chrome()

In [123]:
task_type = 'taskCID' # can be 'taskRID' for realID
name = 'First'
last_name = 'Last'
phone_number = '6280000000'

In [124]:
driver.get("https://www.dmv.ca.gov/wasapp/foa/clear.do?goTo=officeVisit&localeName=en")

In [125]:
task_selector = driver.find_element_by_id('one_task')
task_selector.click()

In [126]:
task_type_selector = driver.find_element_by_id(task_type)
if not task_type_selector.is_selected():
    task_type_selector.click() 

In [127]:
name_field = driver.find_element_by_name('firstName')
name_field.clear()
name_field.send_keys(name)
lastname_field = driver.find_element_by_name('lastName')
lastname_field.clear()
lastname_field.send_keys(last_name)

In [128]:
phone_area_field = driver.find_element_by_name('telArea')
phone_area_field.send_keys(phone_number)

In [ ]:
results = {}

In [159]:
target_office_names = ['PLEASANTON',
                'OAKLAND CLAREMONT',
                'OAKLAND COLISEUM',
                'FREMONT',
                'DALY CITY',
                'HAYWARD',
                'SAN FRANCISCO',
                'EL CERRITO',
                'CONCORD',
                'WALNUT CREEK',
                'PITTSBURG']

In [160]:
selector_field = driver.find_element_by_name('officeId')
selector = Select(selector_field)
office_names = selector_field.text.split('\n')
office_names.insert(0, '<empty>')
offices_count = len(selector.options)
for idx, office_name in enumerate(office_names):
#    print(str(idx) + ": " + office_name)
    if (not office_name in target_office_names):
        continue

    try:
        selector_field = driver.find_element_by_name('officeId')
        Select(selector_field).select_by_index(idx)
        submit = driver.find_element_by_class_name('btn-primary')
        submit.click()

        table_rows = driver.find_elements_by_css_selector('tbody > tr > td > p.no-margin-bottom')
        address = 'not available'
        
        if len(table_rows)>0:
            address = table_rows[0].text
        if ('not available' in address) or ('no appointment is available' in address):
            print("Skipping {idx}({name}): {address}".format(idx=idx, name=office_name, address=address))
        else:
            datetime_str = table_rows[2].text
            datetime_val = dateparser.parse(datetime_str)
            results[address] = datetime_val

        driver.back()
    except:
        print("Problem processing element " + str(idx))
        raise

Skipping 32(EL CERRITO): Sorry, no appointment is available for the date and/or time entered. Please make another selection.


In [161]:
selector_field = driver.find_element_by_name('officeId')
Select(selector_field).select_by_index(1)

In [162]:
sorted([date.isoformat() + ' ' + address for address, date in results.items()])

['2018-08-22T14:40:00 HAYWARD\n150 Jackson Street\nHAYWARD, CA',
 '2018-08-23T10:20:00 FREMONT\n4287 Central Ave.\nFREMONT, CA',
 '2018-08-23T11:00:00 CONCORD\n2070 Diamond Boulevard\nCONCORD, CA',
 '2018-08-27T08:40:00 OAKLAND COLISEUM\n501 85th Avenue\nOAKLAND COLISEUM, CA',
 '2018-08-28T12:50:00 OAKLAND CLAREMONT\n5300 Claremont Ave.\nOAKLAND CLAREMONT, CA',
 '2018-08-29T10:40:00 DALY CITY\n1500 Sullivan Avenue\nDALY CITY, CA',
 '2018-08-30T11:20:00 SAN FRANCISCO\n1377 Fell St.\nSAN FRANCISCO, CA',
 '2018-09-04T09:20:00 WALNUT CREEK\n1910 N. Broadway\nWALNUT CREEK, CA',
 '2018-09-04T11:00:00 PITTSBURG\n1399 Buchanan Road\nPITTSBURG, CA',
 '2018-09-06T14:20:00 PLEASANTON\n6300 W. Las Positas Blvd.\nPLEASANTON, CA']

In [ ]:
driver.close()